# Overview
- Functools
- itertools
    - __iter__() and __next__()
- Generator functions
    - yield statement

## Functools
- functions that act on or return other functions
    - reduce()
    - generic function (multiple functions implementing the same operation for different types)
        - PEP443
        -  @singledispatchmethod for classes
    - cache and lru_cache decorator for cpu-intense computation that do not change often (memoization)
    - total_ordering - eq() and lt() or gt() etc. -> rest is computed
    - wraps to create own decorator
        - options to pass params to decorator function (not decorated function) (no example)
        - option to stack params (no example)

In [10]:
import functools
lisi = [1,2,3,4,5]

#def reduce(function, iterable, initializer=None):
# calculates ((((1+2)+3)+4)+5). 
# The left argument, x, is the accumulated value and 
# the right argument, y, is the update value from the iterable
print(functools.reduce(lambda x, y: x+y, lisi))
print(functools.reduce(lambda x, y: x+y, lisi, 5))

15
20


In [17]:
from functools import singledispatch

@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

@fun.register
def _(arg: int, verbose=False):
   if verbose:
       print("Strength in numbers, eh?", end=" ")
   print(arg)

@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

fun('hello', verbose=True)
fun(10, verbose=True)
fun([1,'huhu', '20'],verbose=True)


Let me just say, hello
Strength in numbers, eh? 10
Enumerate this:
0 1
1 huhu
2 20


In [3]:
# Memoization
# maxsize = num function calls
# cache decorator is smaller and faster
from functools import lru_cache, cache
import time

def count_vowels_no_cache(sentence: str):
    sentence = sentence.casefold()
    return sum(sentence.count(vowel) for vowel in 'aeiou')

@lru_cache(maxsize=100)
def count_vowels_lru(sentence: str):
    sentence = sentence.casefold()
    return sum(sentence.count(vowel) for vowel in 'aeiou')

@cache
def count_vowels_cache(sentence: str):
    sentence = sentence.casefold()
    return sum(sentence.count(vowel) for vowel in 'aeiou')

begin = time.time()
print(count_vowels_no_cache("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels without caching: ", end-begin)

begin = time.time()
print(count_vowels_lru("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching lru 1st run: ", end-begin)

begin = time.time()
print(count_vowels_lru("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching lru 2st run: ", end-begin)

begin = time.time()
print(count_vowels_cache("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching 1st run: ", end-begin)

begin = time.time()
print(count_vowels_cache("This is a nice caching function!"))
end = time.time()
print("Time taken to count vowels with caching 1st run: ", end-begin)

10
Time taken to count vowels without caching:  0.0004470348358154297
10
Time taken to count vowels with caching lru 1st run:  0.004292964935302734
10
Time taken to count vowels with caching lru 2st run:  0.00013303756713867188
10
Time taken to count vowels with caching 1st run:  0.00022602081298828125
10
Time taken to count vowels with caching 1st run:  0.000225067138671875


In [7]:
# Total Ordering
# Performance impact! define all 6 manually if to slow
# eq() and lt() defined manually
# => python computes gt(), ge(), le(), noteq() 
from functools import total_ordering

@total_ordering
class Actor:
    def __init__(self, lastname, firstname):
        self.firstname = firstname
        self.lastname = lastname
    def __eq__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) ==
                (other.lastname.lower(), other.firstname.lower()))
    def __lt__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) <
                (other.lastname.lower(), other.firstname.lower()))


actor1 = Actor("Alfred","James")
actor2 = Actor("Haddock","Captain")

# Auto computed
print(actor2 > actor1) 
print(actor2 >= actor1)
print(actor2 != actor1)

True
True
True


In [31]:
# Wraps
# 1. Decorator is called and returns wrapper
# 2. Wrapper function is called, decorates + passes args to wrapped function
# 3. Inner function is called and result returned
from functools import wraps
import time

# timing = decorator
def timing(fn):
    print("Decorator timing begin...")
    @wraps(fn)
    # args and kwargs passed on to decorated function 
    def wrapper(*args, **kwargs):
        print("Wrapper timing begin...")
        start_time = time.perf_counter()
        # inner function called
        fn_result = fn(*args, **kwargs)
        end_time = time.perf_counter()
        time_duration = end_time - start_time
        print("Function {} took: {} s".format(fn.__name__, time_duration))
        print("Wrapper timing end...")
        return fn_result
    print("Decorator timing end...")
    return wrapper

def fn_no_params():
    return "gugugag"

def fn_with_params(a, b, c=None):
    return a + b if c else 0

@timing
def fn_with_params_dec(a, b, c=None):
    return a + b if c else 0


# No need for params, function could be passed directly
# Wrapper object is returned
decorated_fun = timing(fn_no_params)

# Wrapper object is called now
print(decorated_fun())

# Works thanks to *args and **kwargs
decorated_fun2 = timing(fn_with_params)
print(decorated_fun2(a=10, b=20, c=True))

# Decorator style
print(fn_with_params_dec(a=10, b=20, c=True))



Decorator timing begin...
Decorator timing end...
Decorator timing begin...
Decorator timing end...
Wrapper timing begin...
Function fn_no_params took: 1.213999894389417e-06 s
Wrapper timing end...
gugugag
Decorator timing begin...
Decorator timing end...
Wrapper timing begin...
Function fn_with_params took: 2.41899988395744e-06 s
Wrapper timing end...
30
Wrapper timing begin...
Function fn_with_params_dec took: 1.725999936752487e-06 s
Wrapper timing end...
30


## Itertools
- Iterable: Something that can be looped over, e.g. list, tuples, dicts, strings, files, generator
    - How can we tell if it can be looped over or not? How can we tell if it is iterable?
        - Answer: dunder __iter__() method implemented
        - dunder __iter__() method returns an iterator object (that implements dunder __next__() method)
- Iterator: Object with a state that remembers where it is during an iteration and now how to get next value
    - dunder __iter__() method returns an iterator
    - dunder __next__() method to get next value
    - StopIteration exception if no more next values
    - Can only go forward, not back etc.
    - Make own classes iterable
    - Dont need to end, can go forever
- Generator: Create easy to use iterators
    - Look like normal function but do not return a result but yield a value
    - Yield a value = keep state until generator runs again and yields next value
    - dunder __iter__() and dunder __next__() created automatically
- Itertools: Contain many commonly used iterators + functions to combine iterators
    - zip() to combine 2 iterables
        - 1st value of 1st iterable combined with 1st value of 2nd iterable
        - 2nd value of 1st iterable combined with 2nd value of 2nd interable
        - returns iterator that can be looped over
        - ends on shortes iterable (e.g. list1 has 5 entries and list2 has 7 entries, ends after 5)
            - zip_longest() to pair for longest iterable and pairs with None
    - Examples
        - Infinitely: count(), cycle(), repeat()
        - Finitely: repeat(value, times=n) -> StopIteration
    - map(): Function + iterables as arguments
        - Will call the function with each iterable elements as arguments
    - starmap(): Function + list of tuples that have the arguments already paired together
        - will call the function for each of the tuples with the content of the tuple as argument
- Thanks @CoreySchafer

In [43]:
nums = [1, 2, 3]

# Check if dunder iter() method implemented = iterable
# Not an iterator as duncer next() not implemented
print(dir(nums))

for num in nums:
    print(num)

# List object not an iterator 
# print(next(nums))

# For loop gets this (iterator) in the background
iterator_nums = nums.__iter__() 
print(type(iterator_nums))

# iter() function calls __iter__() in the background
iterator_nums2 = iter(nums) # same as before
print(type(iterator_nums2))

# Check if dunder iter() method implemented = iterable > True
# Check if duncer next() method implemented = iterator > True
print(dir(iterator_nums))

# Next() to iterate through the values
print(next(iterator_nums))
print(next(iterator_nums))
print(next(iterator_nums))

# StopIteration exception > Run out of value
# For-Loop knows how to handle this
# print(next(iterator_nums)) 

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']
1
2
3
<class 'list_iterator'>
<class 'list_iterator'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']

In [48]:
# Make class iterable
class MyRange:
    def __init__(self, start, end):
        self.value = start
        self.end = end

    def __iter__(self):
        # iter function needs to return iterator object
        # = object that has a next() method
        # If next method implemented, can return self
        return self

    def __next__(self):
        if self.value >= self.end:
            raise StopIteration
        current = self.value
        self.value += 1
        return current

numbers = MyRange(1, 10)

# Manual call
print(next(numbers))

# Via for-Loop
for num in numbers:
    print(num)

<class '__main__.MyRange'>
1
2
3
4
5
6
7
8
9


In [55]:
# Generator function
def my_range(start, end):
    current = start
    # End of while loop = StopIteration exception
    while current < end:
        # Returns current value and waits for subsequent next() call
        yield current
        current += 1

numbers = my_range(1,10)
# Via next or using a for loop
print(next(numbers))
print(next(numbers))
print(next(numbers))
print(next(numbers))
print(next(numbers))
print(next(numbers))
print(next(numbers))
print(next(numbers))
print(next(numbers))
# StopIteration Exception
# print(next(numbers))

1
2
3
4
5
6
7
8
9


In [72]:
# Itertools 1 - Sample function (count) from package
import itertools

# start at 0 and count up by 1
# infinitely
counter = itertools.count() 
print(next(counter))
print(next(counter))

counter2 = itertools.count(start=5, step=2)
print(next(counter2))
print(next(counter2))

0
1
5
7


In [62]:
# Itertools 2 - working with the data
import itertools

data = [100, 200, 300, 400]

# Pair up data with an index (100 associated with 0, 200 with 1 etc.)
daily_data = zip(itertools.count(), data)
print(daily_data)

# Could convert zip object to list
# daily_data_list = list(daily_data)
# print(daily_data_list)
for entry in daily_data:
    print(entry)

[(0, 100), (1, 200), (2, 300), (3, 400)]


In [75]:
# Itertools 3 - Cycling through some values
# E.g. Switch on/off
import itertools

counter = itertools.cycle([1, 2, 3])
light_switch = itertools.cycle(('On','Off'))

print(next(counter))
print(next(counter))
print(next(counter))
print(next(counter))
print(next(counter))
print(next(counter))

print(next(light_switch))
print(next(light_switch))
print(next(light_switch))
print(next(light_switch))


1
2
3
1
2
3
On
Off
On
Off


In [86]:
# Itertools 4 - Combining with map() function to generate squares of num 1 to 10
import itertools

# Map takes a function and takes multiple iterables and passes the values of these iterables
#  to the function until shortest list of arguments has run through all values
#  pow(0, 2), pow(1, 2), pow(2, 2), pow(3, 2) etc.
squares = map(pow, range(10), itertools.repeat(2))
print(list(squares))

# Takes arguments that are already paired together as tuples
#data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
data = [val for val in range(10)]
print(data)
zipper = list(zip(data, itertools.repeat(2)))
print(zipper)
#squares2 = itertools.starmap(pow, [(0, 2)])
squares2 = itertools.starmap(pow, zipper)
print(list(squares2))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2)]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
